In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import json
import warnings
from unidecode import unidecode

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/F1-Fantasy-Predictor/'
elif platform == "darwin":
    path = '~/Documents/GitHub/F1-Fantasy-Predictor/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
years =  list(range(2014, 2023 + 1))

In [3]:
races = {
        'season': [],
        'round': [],
        'circuit_id': [],
        'country': [],
        'lat': [],
        'long': [],
        'url': []
        }

for year in years:
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        races['season'].append(int(item['season']))
        races['round'].append(int(item['round']))
        races['circuit_id'].append(item['Circuit']['circuitId'])
        races['country'].append(item['Circuit']['Location']['country'])    
        races['lat'].append(float(item['Circuit']['Location']['lat']))
        races['long'].append(float(item['Circuit']['Location']['long']))
        races['url'].append(item['url'])

races = pd.DataFrame(races)
races.head()

,season,round,circuit_id,country,lat,long,url
0,2014,1,albert_park,Australia,-37.84970,144.96800,http://en.wikipedia.org/wiki/2014_Australian_G...
1,2014,2,sepang,Malaysia,2.76083,101.73800,http://en.wikipedia.org/wiki/2014_Malaysian_Gr...
2,2014,3,bahrain,Bahrain,26.03250,50.51060,http://en.wikipedia.org/wiki/2014_Bahrain_Gran...
3,2014,4,shanghai,China,31.33890,121.22000,http://en.wikipedia.org/wiki/2014_Chinese_Gran...
4,2014,5,catalunya,Spain,41.57000,2.26111,http://en.wikipedia.org/wiki/2014_Spanish_Gran...


In [4]:
### Query API

schedules = {
        'season': [],
        'round': [],
        'circuit_id': [],
        'date': []
        }

for year in years:

    url = 'http://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        schedules['season'].append(int(item['season']))
        schedules['round'].append(int(item['round']))
        schedules['circuit_id'].append(item['Circuit']['circuitId'])
        schedules['date'].append(datetime.strptime(item['date']+ 'T' +item['time'], '%Y-%m-%dT%H:%M:%SZ'))

schedules = pd.DataFrame(schedules)
schedules.head()

,season,round,circuit_id,date
0,2014,1,albert_park,2014-03-16 06:00:00
1,2014,2,sepang,2014-03-30 08:00:00
2,2014,3,bahrain,2014-04-06 15:00:00
3,2014,4,shanghai,2014-04-20 07:00:00
4,2014,5,catalunya,2014-05-11 12:00:00


In [5]:
race_schedules = pd.merge(races, schedules, how='inner', on=['season', 'round', 'circuit_id'])

race_schedules.to_csv(path+'data/races.csv', index=False)

OSError: Cannot save file into a non-existent directory: '/Users/oliverjcarter/Documents/GitHub/F1-Fantasy-Predictor/data'

In [ ]:
races = pd.read_csv(path+'data/races.csv')

rounds = []
for year in np.array(races.season.unique()):
    rounds.append([year, list(races[races.season == year]['round'])])

In [ ]:
# query API
    
results = {
    'season': [],
    'round':[],
    'circuit_id':[],
    'driver': [],
    'date_of_birth': [],
    'nationality': [],
    'constructor': [],
    'grid': [],
    'time': [],
    'status': [],
    'points': [],
    'podium': [],
    'fastest_lap': []
    }

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
        
        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()
        if json['MRData']['RaceTable']['Races'] != []:
            for item in json['MRData']['RaceTable']['Races'][0]['Results']:
                results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
                results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
                results['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
                results['driver'].append(
                    unidecode(item['Driver']['givenName'].lower()) +'_'+ unidecode(item['Driver']['familyName'].lower())
                    )
                results['date_of_birth'].append(item['Driver']['dateOfBirth'])
                results['nationality'].append(item['Driver']['nationality'])
                results['constructor'].append(item['Constructor']['constructorId'])
                results['grid'].append(int(item['grid']))

                try:
                    results['time'].append(int(item['Time']['millis']))
                except:
                    results['time'].append(None)

                results['status'].append(item['status'])
                results['points'].append(float(item['points']))
                results['podium'].append(int(item['position']))

                try:
                    results['fastest_lap'].append(item['FastestLap']['Time']['time'])
                except:
                    results['fastest_lap'].append(None)


results = pd.DataFrame(results)
results.head()

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,time,status,points,podium,fastest_lap
0,2014,1,albert_park,nico_rosberg,1985-06-27,German,mercedes,3,5578710.0,Finished,25.0,1,1:32.478
1,2014,1,albert_park,kevin_magnussen,1992-10-05,Danish,mclaren,4,5605487.0,Finished,18.0,2,1:33.066
2,2014,1,albert_park,jenson_button,1980-01-19,British,mclaren,10,5608737.0,Finished,15.0,3,1:32.917
3,2014,1,albert_park,fernando_alonso,1981-07-29,Spanish,ferrari,5,5613994.0,Finished,12.0,4,1:33.186
4,2014,1,albert_park,valtteri_bottas,1989-08-28,Finnish,williams,15,5626349.0,Finished,10.0,5,1:32.616


In [ ]:
results.to_csv(path+'data/results.csv', index=False)

In [ ]:
driver_standings = {
    'season': [],
    'round':[],
    'driver': [],
    'driver_points': [],
    'driver_wins': [],
    'driver_standings_pos': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        if json['MRData']['StandingsTable']['StandingsLists'] != []:
            for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
                driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
                driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))    
                driver_standings['driver'].append(
                    unidecode(item['Driver']['givenName'].lower()) +'_'+ unidecode(item['Driver']['familyName'].lower())
                    )
                driver_standings['driver_points'].append(float(item['points']))
                driver_standings['driver_wins'].append(int(item['wins']))
                driver_standings['driver_standings_pos'].append(int(item['position']))

driver_standings = pd.DataFrame(driver_standings)
driver_standings.tail(21)

,season,round,driver,driver_points,driver_wins,driver_standings_pos
3908,2023,1,oscar_piastri,0.0,0,20
3909,2023,2,max_verstappen,44.0,1,1
3910,2023,2,sergio_perez,43.0,1,2
3911,2023,2,fernando_alonso,30.0,0,3
3912,2023,2,carlos_sainz,20.0,0,4
3913,2023,2,lewis_hamilton,20.0,0,5
3914,2023,2,george_russell,18.0,0,6
3915,2023,2,lance_stroll,8.0,0,7
3916,2023,2,charles_leclerc,6.0,0,8
3917,2023,2,valtteri_bottas,4.0,0,9


In [ ]:
driver_standings.to_csv(path+'data/driver_standings.csv', index=False)

In [ ]:
constructor_rounds = rounds

constructor_standings = {'season': [],
                    'round':[],
                    'constructor': [],
                    'constructor_points': [],
                    'constructor_wins': [],
                   'constructor_standings_pos': []
                   }

for n in list(range(len(constructor_rounds))):
    for i in constructor_rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(constructor_rounds[n][0], i))
        json = r.json()

        if json['MRData']['StandingsTable']['StandingsLists'] != []:
            for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
                constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
                constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
                constructor_standings['constructor'].append(item['Constructor']['constructorId'])
                constructor_standings['constructor_points'].append(float(item['points']))
                constructor_standings['constructor_wins'].append(int(item['wins']))
                constructor_standings['constructor_standings_pos'].append(int(item['position']))
            
constructor_standings = pd.DataFrame(constructor_standings)
constructor_standings.tail(10)

,season,round,constructor,constructor_points,constructor_wins,constructor_standings_pos
1869,2023,2,red_bull,87.0,2,1
1870,2023,2,aston_martin,38.0,0,2
1871,2023,2,mercedes,38.0,0,3
1872,2023,2,ferrari,26.0,0,4
1873,2023,2,alpine,8.0,0,5
1874,2023,2,alfa,4.0,0,6
1875,2023,2,haas,1.0,0,7
1876,2023,2,williams,1.0,0,8
1877,2023,2,alphatauri,0.0,0,9
1878,2023,2,mclaren,0.0,0,10


In [ ]:
constructor_standings.to_csv(path+'data/constructor_standings.csv', index=False)